# Covermap Comparison

**Author:** Adam Yang (ayang115@umd.edu)

**Description:** Compares datasets against test sets

**Crop Maps Used:** Harvest Togo & Kenya, Copernicus Land Cover, ESA CCI Land Cover Africa, GLAD Global Cropland Extent

In [135]:
import pandas as pd
import numpy as np
import geopandas as gdp
import os
import geemap
from pathlib import Path
from sklearn.metrics import classification_report 

In [136]:
import ee

ee.Authenticate()

ee.Initialize()


Successfully saved authorization token.


## **Section 0** - Setup

**Functions**

In [137]:
# Remaps classes to crop/noncrop 
def map_values(val, value_for_crop):
    if val == value_for_crop:
        return 1
    else:
        return 0

In [138]:
# Function used in map function to extract from feature collection
def raster_extraction(image, resolution, f_collection):
    feature = image.sampleRegions(
        collection = f_collection,
        scale = resolution
    )
    return feature

In [139]:
# Convert sklearn classification report dict to 
def report_to_row(dataset, report, df):
    new_report = pd.DataFrame(data = {
        "dataset": dataset, 
        "accuracy": report["accuracy"], 
        "crop_f1": report["1"]["f1-score"], 
        "crop_support": report["1"]["support"], 
        "noncrop_support": report["0"]["support"], 
        "crop_precision": report["1"]["precision"], 
        "crop_recall": report["1"]["recall"], 
        "noncrop_precision": report["0"]["precision"], 
        "noncrop_recall": report["0"]["recall"]
        }, index=[0])
    
    return pd.concat([df, new_report])

In [140]:
# Creates ee.Feature from longitude and latitude coordinates from a dataframe
def create_point(row):
    geom = ee.Geometry.Point(row["lon"], row["lat"])
    prop = dict(row)

    return ee.Feature(geom, prop)

**Retrieve Test Data**

In [141]:
TEST_COUNTRIES = ["Kenya", "Togo", "Tanzania_CEO_2019"]
DATA_PATH = "../data/datasets/"

In [142]:
datasets_path = Path(DATA_PATH).glob("*")

target_paths = [p for p in datasets_path if p.stem in TEST_COUNTRIES]

In [143]:
test_data = pd.DataFrame(columns=["lat", "lon", "test_class", "ee_pts", "country"])

test_set = []
for p in target_paths:
    # Set dict key name
    key = p.stem

    # Read in data and extract test values and points 
    df = pd.read_csv(p)
    df = df.loc[df["subset"] == "testing"]
    df = df[["lat", "lon", "class_probability"]]

    # Create earth engine geometry points
    df["ee_pts"] = df.apply(create_point, axis=1)

    # Recast points as 1 or 0 (threshold = 0.5)
    df["test_class"] = df["class_probability"].apply(lambda x: 1 if x>=0.5 else 0)

    df["country"] = key

    test_set.append(df)

test_data = pd.concat(test_set)

In [144]:
test_data.reset_index(inplace=True)
test_data.drop("index", axis=1, inplace=True)

In [145]:
test_data.head()

,lat,lon,class_probability,ee_pts,test_class,country
0,0.725122,34.386282,0.0,"ee.Feature({\n ""functionInvocationValue"": {\n...",0,Kenya
1,0.621939,34.466496,0.0,"ee.Feature({\n ""functionInvocationValue"": {\n...",0,Kenya
2,0.459661,34.090158,0.0,"ee.Feature({\n ""functionInvocationValue"": {\n...",0,Kenya
3,0.226497,34.054859,0.0,"ee.Feature({\n ""functionInvocationValue"": {\n...",0,Kenya
4,0.096834,33.951959,0.0,"ee.Feature({\n ""functionInvocationValue"": {\n...",0,Kenya


**Create dataframe for results**

In [192]:
results = {}

for p in target_paths:
    key = p.stem
    results[key] = pd.DataFrame(columns=["dataset", "accuracy", "crop_f1", "crop_support", "noncrop_support", 
        "crop_precision", "crop_recall", "noncrop_precision", "noncrop_recall"])


In [147]:
test_coll = ee.FeatureCollection(test_data["ee_pts"].tolist())

## **Section 1** - Harvest Data

**Harvest Togo**

In [148]:
harvest_togo = ee.Image("projects/sat-io/open-datasets/nasa-harvest/togo_cropland_binary").sampleRegions(collection=test_coll)

In [149]:
h_togo_sampled = geemap.ee_to_gdf(harvest_togo)
h_togo_sampled.head()

,geometry,b1,class_probability,lat,lon
0,None,0,0.00,9.875907,1.172471
1,None,0,0.25,9.180105,1.374695
2,None,0,0.00,9.275314,1.196385
3,None,0,0.00,7.346575,0.705167
4,None,0,0.25,6.777337,0.629817


In [150]:
test_data["harvest_togo"] = pd.merge(test_data, h_togo_sampled, on=["lat", "lon"], how="left")["b1"]

**Harvest Kenya**

In [151]:
harvest_kenya = ee.Image("projects/sat-io/open-datasets/nasa-harvest/kenya_cropland_binary").sampleRegions(collection=test_coll)

In [152]:
h_kenya_sampled = geemap.ee_to_gdf(harvest_kenya)
h_kenya_sampled.head()

,geometry,b1,class_probability,lat,lon
0,None,1,0.0,0.725122,34.386282
1,None,1,0.0,0.621939,34.466496
2,None,0,0.0,0.226497,34.054859
3,None,0,0.0,0.096834,33.951959
4,None,0,0.0,0.032072,33.968469


In [153]:
test_data["harvest_kenya"] = pd.merge(test_data, h_kenya_sampled, on=["lat", "lon"], how="left")["b1"]

**Harvest Tanzania**

In [154]:
harvest_tanzania = ee.Image("users/adadebay/Tanzania_cropland_2019").sampleRegions(collection=test_coll)

In [155]:
h_tanzania_sampled = geemap.ee_to_gdf(harvest_tanzania)
h_tanzania_sampled["b1"] = h_tanzania_sampled["b1"].apply(lambda x: 1 if x>=0.5 else 0) 
h_tanzania_sampled.head()

,geometry,b1,class_probability,lat,lon
0,None,0,0.0,-1.121763,33.847992
1,None,0,0.0,-2.979751,37.342422
2,None,0,0.0,-3.058859,37.347080
3,None,0,0.0,-3.067081,37.380394
4,None,0,0.0,-3.069923,37.362679


In [156]:
h_tanzania_sampled.to_csv("test.csv")

In [157]:
test_data["harvest_tanzania"] = pd.merge(test_data, h_tanzania_sampled, on=["lat", "lon"], how="left")["b1"]

## **Section 2** - Copernicus, ESA, GLAD

In [158]:
# Create earth engine points for section 2
test_coll = ee.FeatureCollection(test_data["ee_pts"].tolist())

**Copernicus Land Cover** ([Earth Engine](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global#description))

In [159]:
# Load copernicus data
copernicus = ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V-C3/Global")
cop_results = copernicus.select("discrete_classification").filterDate("2019-01-01", "2020-01-01").map(lambda x: raster_extraction(x, 100, test_coll)).flatten()

In [160]:
cop_sampled = geemap.ee_to_gdf(cop_results)
cop_sampled["cop_class"] = cop_sampled["discrete_classification"].apply(lambda x: map_values(x, 40))
cop_sampled.head()

,geometry,class_probability,discrete_classification,lat,lon,cop_class
0,None,0.0,126,0.725122,34.386282,0
1,None,0.0,122,0.621939,34.466496,0
2,None,0.0,50,0.459661,34.090158,0
3,None,0.0,40,0.226497,34.054859,1
4,None,0.0,20,0.096834,33.951959,0


In [161]:
test_data["cop"] = pd.merge(test_data, cop_sampled, on=["lat", "lon"], how="left")["cop_class"]

**ESA World Cover** ([Earth Engine](https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100)) 

In [162]:
esa = ee.ImageCollection("ESA/WorldCover/v100")
esa_results = esa.filterBounds(test_coll).map(lambda x: raster_extraction(x, 10, test_coll)).flatten()

In [163]:
esa_sampled = geemap.ee_to_gdf(esa_results)
esa_sampled["esa_class"] = esa_sampled["Map"].apply(lambda x: map_values(x, 40))
esa_sampled.head()

,geometry,Map,class_probability,lat,lon,esa_class
0,None,20,0.0,0.725122,34.386282,0
1,None,20,0.0,0.621939,34.466496,0
2,None,50,0.0,0.459661,34.090158,0
3,None,30,0.0,0.226497,34.054859,0
4,None,30,0.0,0.096834,33.951959,0


In [164]:
test_data["esa"] = pd.merge(test_data, esa_sampled, on=["lat", "lon"], how="left")["esa_class"]

**GLAD Global** ([Earth Engine](https://glad.earthengine.app/view/global-cropland-dynamics))

In [165]:
glad = ee.ImageCollection("users/potapovpeter/Global_cropland_2019")
glad_results = glad.filterBounds(test_coll).map(lambda x: raster_extraction(x, 30, test_coll)).flatten()

In [166]:
glad_sampled = geemap.ee_to_gdf(glad_results)
glad_sampled.head()

,geometry,b1,class_probability,lat,lon
0,None,1,0.0,0.725122,34.386282
1,None,0,0.0,0.621939,34.466496
2,None,0,0.0,0.459661,34.090158
3,None,0,0.0,0.226497,34.054859
4,None,0,0.0,0.096834,33.951959


In [167]:
test_data['glad'] = pd.merge(test_data, glad_sampled, on=["lat", "lon"], how="left")["b1"]

In [168]:
test_data

,lat,lon,class_probability,ee_pts,test_class,country,harvest_togo,harvest_kenya,harvest_tanzania,cop,esa,glad
0,0.725122,34.386282,0.00,"ee.Feature({\n ""functionInvocationValue"": {\n...",0,Kenya,NaN,1.0,NaN,0,0.0,1
1,0.621939,34.466496,0.00,"ee.Feature({\n ""functionInvocationValue"": {\n...",0,Kenya,NaN,1.0,NaN,0,0.0,0
2,0.459661,34.090158,0.00,"ee.Feature({\n ""functionInvocationValue"": {\n...",0,Kenya,NaN,NaN,NaN,0,0.0,0
3,0.226497,34.054859,0.00,"ee.Feature({\n ""functionInvocationValue"": {\n...",0,Kenya,NaN,0.0,NaN,1,0.0,0
4,0.096834,33.951959,0.00,"ee.Feature({\n ""functionInvocationValue"": {\n...",0,Kenya,NaN,0.0,NaN,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3889,6.472810,1.502436,0.75,"ee.Feature({\n ""functionInvocationValue"": {\n...",1,Togo,1.0,NaN,NaN,1,0.0,0
3890,11.037647,0.048311,1.00,"ee.Feature({\n ""functionInvocationValue"": {\n...",1,Togo,1.0,NaN,NaN,1,1.0,1
3891,9.011538,1.493877,0.00,"ee.Feature({\n ""functionInvocationValue"": {\n...",0,Togo,0.0,NaN,NaN,1,0.0,0
3892,9.138645,1.391047,0.00,"ee.Feature({\n ""functionInvocationValue"": {\n...",0,Togo,0.0,NaN,NaN,0,0.0,0


## **Evaluation Results**

**Compute results**

In [193]:
for country, df in test_data.groupby("country"):
    for dataset in ["cop", "esa", "glad", "harvest_togo", "harvest_kenya", "harvest_tanzania"]:
        # If country is non-empty
        if not pd.isnull(df[dataset]).all() or not np.isnan(np.unique(df[dataset])[1]):
            print(country + ": " + dataset)
            # Remove na values
            temp = df[["test_class", dataset]].dropna()
            if len(temp) > 10:
                report = classification_report(temp["test_class"], temp[dataset], output_dict= True)     

            results[country] = report_to_row(dataset, report, results[country])

Kenya: cop
Kenya: esa
Kenya: glad
Kenya: harvest_kenya
Kenya: harvest_tanzania
Tanzania_CEO_2019: cop
Tanzania_CEO_2019: esa
Tanzania_CEO_2019: glad
Tanzania_CEO_2019: harvest_kenya
Tanzania_CEO_2019: harvest_tanzania
Togo: cop
Togo: esa
Togo: glad
Togo: harvest_togo


**Kenya**

In [194]:
results["Kenya"]

,dataset,accuracy,crop_f1,crop_support,noncrop_support,crop_precision,crop_recall,noncrop_precision,noncrop_recall
0,cop,0.913148,0.937824,571,258,0.925043,0.950963,0.884298,0.829457
0,esa,0.449940,0.341040,571,258,0.975207,0.206655,0.360169,0.988372
0,glad,0.849216,0.885636,571,258,0.927203,0.847636,0.716612,0.852713
0,harvest_kenya,0.952722,0.968054,520,178,0.974659,0.961538,0.891892,0.926966
0,harvest_tanzania,0.952722,0.968054,520,178,0.974659,0.961538,0.891892,0.926966


**Togo**

In [195]:
results["Togo"]

,dataset,accuracy,crop_f1,crop_support,noncrop_support,crop_precision,crop_recall,noncrop_precision,noncrop_recall
0,cop,0.695775,0.571429,152,203,0.720000,0.473684,0.686275,0.862069
0,esa,0.740845,0.606838,152,203,0.865854,0.467105,0.703297,0.945813
0,glad,0.707042,0.559322,152,203,0.785714,0.434211,0.682657,0.911330
0,harvest_togo,0.771831,0.736156,152,203,0.729032,0.743421,0.805000,0.793103


**Tanzania**

In [196]:
results["Tanzania_CEO_2019"]

,dataset,accuracy,crop_f1,crop_support,noncrop_support,crop_precision,crop_recall,noncrop_precision,noncrop_recall
0,cop,0.783764,0.696058,1174,1536,0.889920,0.571550,0.742843,0.945964
0,esa,0.772934,0.668511,1174,1464,0.954186,0.514480,0.715711,0.980191
0,glad,0.774539,0.659610,1174,1536,0.953301,0.504259,0.721398,0.981120
0,harvest_kenya,0.647975,0.443350,154,167,0.918367,0.292208,0.599265,0.976048
0,harvest_tanzania,0.762060,0.711006,765,769,0.901606,0.586928,0.694981,0.936281
